In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
import warnings

%matplotlib inline
plt.style.use('fivethirtyeight')
warnings.simplefilter(action="ignore", category=FutureWarning)

# Data Cleaning for Consistency

Import the Peru data file into the notebook

In [ ]:
ylives = Table.read_table(....)
ylives

### Sex Variables Cleaning
Check the values of each of the sex variables

In [ ]:
ylives.group('sex_5y')

In [ ]:
ylives.group('...')

In [ ]:
#check 12 years here

Check if the sexes match for ages 5 & 8.

In [ ]:
test1 = ylives.where(ylives['sex_5y'] == ylives['sex_8y'])
test1

Check if the sexes match for ages 5 & 8 and for ages 8 & 12.

In [ ]:
test2 = np.logical_and(..., ...)
sum(test2)

What does test2 look like?

Check how many are problematic.

In [ ]:
not_equal = np.invert(test2)
sum(not_equal)

Examine the problematic observations

In [ ]:
ylives.where(not_equal)

Code to correct the sex.  When will you not be able to make an educated guess?

In [ ]:
replacement_values = []
rows_to_look_at_further = []

In [ ]:
for i in range(ylives.num_rows):
    if not_equal[i]:
        sexes = [ylives['sex_5y'][i], ylives['sex_8y'][i], ylives['sex_12y'][i]]
        m, f = sexes.count('Male'), sexes.count('Female')
        if m > f:
            replacement_values.append('Male')
        elif f > m:
            replacement_values.append('Female')
        else:
            replacement_values.append(-99)
            rows_to_look_at_further.append(i)
    else:
        replacement_values.append(ylives['sex_5y'][i])

TRANSALATION:

    for each row:
        
        if not all three columns have the same value (which we determined earlier with not_equal):

            make a mini table called sexes
            
            count how many times each sex (m & f) appears

            if more males than females:
                assign value male
            if more females than males:
                assign value female
            if there is one male, one female, and one nan: 
                assign None and add row to table of observations that need to be investigated
        
        if all three columns do have the same value:
            
            add that value to the replacements

Take a look at problem observartions that cannot be fixed.  What is your plan for them?

In [ ]:
ylives.take(rows_to_look_at_further)

Fix the ones that can be fixed

In [ ]:
ylives['sex_5y'] = replacement_values
ylives['...'] = replacement_values
ylives['...'] = ...
ylives

### Relate Variables Cleaning
Try re-doing the above exercise using the relate variables. 
Copy & paste all the cells right here & change from sex to relate.

### Age Variables Cleaning

Now let's think about how to check consistency on the ages.  If we repeat the exercise the same as above, we will have too many errors.  Why?

In [ ]:
#just run this cell - adds a graphing package
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
# converts to pandas & drops nan
agetest = ylives.to_df().dropna(subset=['age_8y', 'age_5y', 'age_12y'])

Note that we are now using the table agetest instead of ylives
Let's examine the age differences between years.

In [ ]:
five_eight = agetest['age_8y'] - agetest['age_5y']
five_eight.value_counts()

This will be easier to see in a graph - note we are dropping the tails

In [ ]:
plt.hist(five_eight, bins=9, range=(-2, 7));

In [ ]:
eight_twelve = test['...'] - test['...']
eight_twelve.value_counts()

In [ ]:
plt.hist(..., bins=11, range=(-2, 9));

In [ ]:
five_twelve = ...
five_twelve.value_counts()

In [ ]:
plt.hist(...);

#### Determining which ages are "ok"
This is a bit of a value judgement.  We can use visual inspection & choose ages based on the histograms, or we can use means and standard deviations.  Beyond 2 standard deviations is very unlikely, so let's flag those.  The next cell calculates means & standard deviations & calculates what portion of observations beyond 2 s.d. are flagged.

In [ ]:
full_array = []
for diff in [five_eight, eight_twelve, five_twelve]:
    sub_array = []
    mu, sigsq = np.mean(diff), np.std(diff)
    print('Mean: {}\nStandard Deviation: {}'.format(mu, sigsq))
    
    for value in diff:
        if abs(mu - value) > 2 * sigsq:
            sub_array.append(False)
        else:
            sub_array.append(True)
    print('Flagged Values:', 1-np.mean(sub_array), '\n')
    full_array.append(sub_array)

We won't code up the rest, but think of the processes we'd have to go through:

For those with a difference outside the window, we have to correct them. 
How do we determine which is the true age to base the corrections on?  
How do we calculate the replacement age?

Reflection: Consider how many corrections we have made for the sex, relate and age variables. Based on the size of the survey, do think this quantity is problematic?